In [4]:
import os

import pandas as pd
import numpy as np

import pydeck as pdk

df = pd.concat([pd.read_csv(f'data/{file}', compression = 'zip') for file in os.listdir('data')])
df['lat'] = np.where(df['lat'].isna(), df['latitude'], df['lat'])
df['lon'] = np.where(df['lon'].isna(), df['longitude'], df['lon'])
df = df[(df['lat'].notna()) & (df['lon'].notna())].copy()
df.drop(['latitude', 'longitude', 'foco_id', 'pais'], axis = 1, inplace = True)
df
# import plotly.colors as pc
# df_summary['color'] = pc.sample_colorscale('RdYlGn', samplepoints = df_summary['estado'] / df_summary['estado'].max(), low = 0, high = df_summary['estado'].max())
# df_summary['color'] = df_summary['color'].str[4:-1].str.split(', ')
# df_summary['color']
# df_summary['r'] = df_summary['color'].str[0].astype(int)
# df_summary['g'] = df_summary['color'].str[1].astype(int)
# df_summary['b'] = df_summary['color'].str[2].astype(int)
# df_summary['color'] = df_summary['color'].apply(lambda val: str([int(v) for v in val]))[0]


,id_bdq,lat,lon,data_pas,estado,municipio,bioma,satelite,numero_dias_sem_chuva,precipitacao,risco_fogo,id_area_industrial,frp
0,9988885.0,-18.51800,-55.02800,2003-05-15 17:05:00,MATO GROSSO DO SUL,RIO VERDE DE MATO GROSSO,Cerrado,NaN,NaN,NaN,NaN,NaN,NaN
1,9988884.0,-17.54200,-48.81500,2003-05-15 17:05:00,GOIÁS,PIRACANJUBA,Cerrado,NaN,NaN,NaN,NaN,NaN,NaN
2,9988882.0,-17.61200,-47.16000,2003-05-15 17:05:00,MINAS GERAIS,GUARDA-MOR,Cerrado,NaN,NaN,NaN,NaN,NaN,NaN
3,9988881.0,-18.38600,-51.91900,2003-05-15 17:05:00,GOIÁS,SERRANÓPOLIS,Cerrado,NaN,NaN,NaN,NaN,NaN,NaN
4,9988879.0,-17.87700,-49.09800,2003-05-15 17:05:00,GOIÁS,MORRINHOS,Cerrado,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
189896,NaN,-10.97751,-50.43589,2023-07-27 18:00:00,TOCANTINS,LAGOA DA CONFUSÃO,Cerrado,AQUA_M-T,46.0,0.0,1.0,0.0,188.2
189897,NaN,-10.97426,-50.44424,2023-07-27 18:00:00,TOCANTINS,LAGOA DA CONFUSÃO,Cerrado,AQUA_M-T,46.0,0.0,1.0,0.0,331.8
189898,NaN,-10.96047,-50.43910,2023-07-27 18:00:00,TOCANTINS,LAGOA DA CONFUSÃO,Cerrado,AQUA_M-T,47.0,0.0,1.0,0.0,282.9
189899,NaN,-10.95722,-50.44749,2023-07-27 18:00:00,TOCANTINS,LAGOA DA CONFUSÃO,Cerrado,AQUA_M-T,47.0,0.0,1.0,0.0,73.3


In [5]:
df['data_pas'] = pd.to_datetime(df['data_pas'])
df['ano'] = df['data_pas'].dt.year
df['id_bdq'] = df['id_bdq'].fillna('a')

In [6]:
df_count = df[['ano', 'estado', 'id_bdq']].groupby(['ano', 'estado']).count().reset_index().pivot_table(
    index = 'ano',
    columns = 'estado',
    values = 'id_bdq'
)

In [7]:
import plotly.graph_objs as go
import plotly.io as pio
import plotly.colors as pc
import plotly.offline

In [8]:
df_count_rel = df_count.copy()

In [9]:
for _, row in df_count_rel.iterrows():
    row /= row.sum()

In [10]:
data = []
layout = go.Layout(barmode = 'stack', template = 'plotly_dark', xaxis_tickmode = 'linear')

colors = dict(zip(
    df_count.columns,
    pc.sample_colorscale('Phase', np.arange(len(df_count.columns)) / len(df_count.columns))
))

for col in df_count.columns:
    data.append(go.Bar(
        x = df_count.index.values,
        y = df_count[col].values,
        name = col,
        # marker_color = colors[col],
        text = [(f'{col}<br>{val:,.0f}' if val > 5_000 else '') for val in df_count[col].values]
        # texttemplate=f'{col}<br>' + '%{y:.3s}'
    ))
    
fig = go.Figure(data = data, layout = layout)
fig.show()

plotly.offline.plot(fig, filename = 'contagem_incendios.html')

'contagem_incendios.html'

In [13]:
data = []
layout = go.Layout(barmode = 'stack', template = 'plotly_dark', xaxis_tickmode = 'linear', yaxis_tickformat = '.1%')

for col in df_count_rel.columns:
    data.append(go.Bar(
        x = df_count_rel.index.values,
        y = df_count_rel[col].values,
        name = col,
        # marker_color = colors[col],
        # text = [(f'{col}<br>{val:,.0f}' if val > 5_000 else '') for val in df_count_rel[col].values]
        texttemplate=f'{col}<br>' + '%{y:.1%}' if col in ('MARANHÃO', 'PARÁ', 'MATO GROSSO', 'BAHIA', 'AMAZONAS', 'TOCANTINS', ) else ''
    ))
    
fig = go.Figure(data = data, layout = layout)
fig.show()

plotly.offline.plot(fig, filename = 'contagem_incendios_rel.html')

'contagem_incendios_rel.html'

In [14]:
from plotly.subplots import make_subplots

In [17]:
sorted(['TOCANTINS', 'MARANHÃO', 'PARÁ', 'MATO GROSSO', 'BAHIA', 'AMAZONAS', 'RONDÔNIA'])

['AMAZONAS',
 'BAHIA',
 'MARANHÃO',
 'MATO GROSSO',
 'PARÁ',
 'RONDÔNIA',
 'TOCANTINS']

In [16]:
data = []
layout = go.Layout(
    barmode = 'stack', template = 'plotly_white', xaxis_tickmode = 'linear',
    yaxis=dict(title='Quantidade de Focos'),
    yaxis2=dict(
        title='Percentual dos Focos nos Principais Estados',
        overlaying='y',
        side='right',
        tickformat = '.0%'
    )
)

# fig = make_subplots()

colors = dict(zip(
    df_count.columns,
    pc.sample_colorscale('Phase', np.arange(len(df_count.columns)) / len(df_count.columns))
))

data.append(go.Bar(
    x = df_count.index.values,
    y = df_count.sum(axis = 1).values,
    name = '# de Focos',
    # marker_color = colors[col],
    texttemplate='%{y:.3s}',
    yaxis='y1'
))

data.append(go.Scatter(
    x = df_count_rel.index.values,
    y = df_count_rel[['TOCANTINS', 'MARANHÃO', 'PARÁ', 'MATO GROSSO', 'BAHIA', 'AMAZONAS', 'RONDÔNIA']].sum(axis = 1).values,
    name = '% do Total de Focos nos Principais Estados',
    # marker_color = colors[col],
    texttemplate='%{y:.1%}',
    yaxis='y2'
))
    
fig = go.Figure(data = data, layout = layout)
fig.show()

plotly.offline.plot(fig, filename = 'contagem_incendios_all.html')

'contagem_incendios_all.html'

In [ ]:
from pydeck.types import String
COLOR_BREWER_BLUE_SCALE = [
    [240, 249, 232],
    [204, 235, 197],
    [168, 221, 181],
    [123, 204, 196],
    [67, 162, 202],
    [8, 104, 172],
]
df['lat_r'] = df['lat'].round(1)
df['lon_r'] = df['lon'].round(1)
df_summary = df[['estado', 'lat_r', 'lon_r']].groupby(['lat_r', 'lon_r']).count().reset_index()

# for map_style in ('light', 'dark', 'road', 'satellite', 'dark_no_labels', 'light_no_labels'):

layer = pdk.Layer(
    "HeatmapLayer",
    df_summary,
    opacity=1,
    get_position=["lon_r", "lat_r"],
    get_weight="estado",
    map_style = 'light'
)

# Set the viewport location
view_state = pdk.ViewState(latitude=-18.51800, longitude=-55.02800, zoom=2, bearing=0, pitch=30)


# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text": "Density: {mean}"})
r.show()
# r.to_html(filename=f'test_heatmap_{map_style}.html', iframe_width='100%', iframe_height=1000)
df['lat_r'] = df['lat'].round(1)
df['lon_r'] = df['lon'].round(1)
df_summary = df[['estado', 'lat_r', 'lon_r']].groupby(['lat_r', 'lon_r']).count().reset_index()

for map_style in ('light', 'dark', 'road', 'satellite', 'dark_no_labels', 'light_no_labels'):

    layer = pdk.Layer(
        "HeatmapLayer",
        df_summary,
        opacity=1,
        get_position=["lon_r", "lat_r"],
        get_weight="estado",
        map_style = map_style
    )

    # Set the viewport location
    view_state = pdk.ViewState(latitude=-18.51800, longitude=-55.02800, zoom=2, bearing=0, pitch=30)


    # Render
    r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text": "Density: {mean}"})
    # r.show()
    r.to_html(filename=f'test_heatmap_{map_style}.html', iframe_width='100%', iframe_height=1000)
df['lat_r'] = df['lat'].round(0)
df['lon_r'] = df['lon'].round(0)
df_summary = df[['estado', 'lat_r', 'lon_r']].groupby(['lat_r', 'lon_r']).count().reset_index()

min_count = df_summary['estado'].min()
max_count = df_summary['estado'].max()
diff = max_count - min_count

color_scheme = f"""[
255 - (255-189) * (estado - {min_count})/{diff}, 
255 - 255 * (estado - {min_count})/{diff}, 
178 - (178-38) * (estado - {min_count})/{diff}
]"""

for map_style in ('light', 'dark', 'road', 'satellite', 'dark_no_labels', 'light_no_labels'):

    layer = pdk.Layer(
        "ColumnLayer",
        df_summary,
        # opacity=1,
        get_position=["lon_r", "lat_r"],
        get_fill_color = color_scheme, #'[255 - estado, 255 - 0.5 * estado, 0]',
        get_elevation="estado",
        elevation_scale=10,
        radius=20_000,
        pickable=True,
        auto_highlight=True,
        map_style = map_style
    )

    # Set the viewport location
    view_state = pdk.ViewState(latitude=-18.51800, longitude=-55.02800, zoom=2, bearing=0, pitch=30)


    # Render
    r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text": "Density: {mean}"})
    r.show()
    # r.to_html(filename=f'test_column_layer_{map_style}.html', iframe_width='100%', iframe_height=1000)